In [1]:
import torch
print(torch.__version__)

2.2.2


In [2]:
import os
os.environ["MKL_THREADING_LAYER"] = "TBB"

In [3]:
import ctgan
dir(ctgan)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


['CTGAN',
 'TVAE',
 '__all__',
 '__author__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__email__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'data_sampler',
 'data_transformer',
 'demo',
 'load_demo',
 'synthesizers']

In [1]:
import pandas as pd
import numpy as np
from ctgan import CTGAN
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from scipy.stats import ks_2samp, chi2_contingency
import time

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


In [2]:
df = pd.read_csv("com_salary.csv")

In [7]:
df.head()

,Name,OttoneuID,FG MajorLeagueID,FG MinorLeagueID,MLB Org,Position(s),Avg Salary,Median Salary,Min Salary,Max Salary,...,POS,ADP,rPTS,PTS,aPOS,Dollars,Adjusted,Cost,PlayerId,value
0,Juan Soto,23717,20123.0,sa906282,NYY,OF,60.31,60.0,34,84,...,OF/DH,10.57,1132.535865,53.429347,21.885623,76.314969,112.280793,NaN,20123,0.000000
1,Mookie Betts,18276,13611.0,sa597889,LAD,2B/SS/OF,58.35,58.0,38,87,...,2B/SS/OF,5.03,1036.172159,40.095221,14.934803,56.030024,82.308998,NaN,13611,0.000000
2,Shohei Ohtani,33600,19755.0,NaN,LAD,Util/SP,56.13,56.0,32,88,...,P/DH,12.73,1038.276653,40.386426,12.178202,53.564628,75.049219,54.0,19755,-0.435372
3,Aaron Judge,18312,15640.0,sa549847,NYY,OF,54.17,54.5,33,70,...,OF/DH,11.16,1066.255227,44.257902,21.885623,67.143525,93.070498,50.0,15640,17.143525
4,Freddie Freeman,5680,5361.0,sa390291,LAD,1B,50.76,51.0,27,74,...,1B,8.81,1062.235872,43.701733,13.809086,58.510819,81.663561,47.0,5361,11.510819



# Assuming `orig_df` is your original DataFrame with the actual data
# Example DataFrame (replace with your actual DataFrame)
orig_df = pd.DataFrame({
    'OttoneuID': [1, 2, 3, 4, 5],
    'FG MajorLeagueID': [1001, 1002, 1003, 1004, 1005],
    'Avg Salary': [10, 20, 30, 40, 50],
    'Median Salary': [15, 25, 35, 45, 55],
    'Min Salary': [5, 10, 15, 20, 25],
    'Max Salary': [20, 30, 40, 50, 60],
    'Last 10': [7, 8, 9, 10, 11],
    'Roster%': [60, 70, 80, 90, 100],
    'ADP': [1, 2, 3, 4, 5],
    'rPTS': [100, 200, 300, 400, 500],
    'PTS': [150, 250, 350, 450, 550],
    'aPOS': [1, 2, 3, 4, 5],
    'Dollars': [10, 20, 30, 40, 50],
    'Adjusted': [5, 10, 15, 20, 25],
    'Cost': [2, 4, 6, 8, 10],
    'value': [8, 16, 24, 32, 40],
    'Name': ['Player1', 'Player2', 'Player3', 'Player4', 'Player5'],
    'FG MinorLeagueID': ['ID1', 'ID2', 'ID3', 'ID4', 'ID5'],
    'MLB Org': ['TeamA', 'TeamB', 'TeamC', 'TeamD', 'TeamE'],
    'Position(s)': ['Pos1', 'Pos2', 'Pos3', 'Pos4', 'Pos5'],
    'Team': ['Team1', 'Team2', 'Team3', 'Team4', 'Team5'],
    'POS': ['POS1', 'POS2', 'POS3', 'POS4', 'POS5'],
    'PlayerId': ['P1', 'P2', 'P3', 'P4', 'P5']
})

In [4]:
orig_df = pd.DataFrame(orig_df)

In [5]:
# Selecting numeric and categorical features
numeric_features = orig_df.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = orig_df.select_dtypes(include=['object', 'category']).columns.tolist()

print(f"Numeric features: {numeric_features}")
print(f"Categorical features: {categorical_features}")

Numeric features: ['OttoneuID', 'FG MajorLeagueID', 'Avg Salary', 'Median Salary', 'Min Salary', 'Max Salary', 'Last 10', 'Roster%', 'ADP', 'rPTS', 'PTS', 'aPOS', 'Dollars', 'Adjusted', 'Cost', 'value']
Categorical features: ['Name', 'FG MinorLeagueID', 'MLB Org', 'Position(s)', 'Team', 'POS', 'PlayerId']


In [6]:
# Define transformers
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('encoder', OneHotEncoder())
])

# Combine transformers into a preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

In [39]:
# Fit and transform the data
prepared_data = preprocessor.fit_transform(orig_df)

# Check if the data is sparse and convert to dense if necessary
if hasattr(prepared_data, "toarray"):
    prepared_data = prepared_data.toarray()

# Check the shape of the prepared data
print(f"Shape of prepared_data: {prepared_data.shape}")

Shape of prepared_data: (5, 51)


In [40]:
# Get the column names from the transformers
numeric_column_names = numeric_features
categorical_column_names = preprocessor.named_transformers_['cat']['encoder'].get_feature_names_out(categorical_features)

# Combine all column names
all_column_names = numeric_column_names + list(categorical_column_names)

# Verify the lengths match
print(f"Number of columns in prepared_data: {prepared_data.shape[1]}")
print(f"Number of column names: {len(all_column_names)}")

Number of columns in prepared_data: 51
Number of column names: 51


In [41]:
# Ensure the lengths match before creating the DataFrame
if prepared_data.shape[1] == len(all_column_names):
    prepared_df = pd.DataFrame(prepared_data, columns=all_column_names)
    print(prepared_df.head())
else:
    raise ValueError("The number of columns in the prepared data does not match the number of column names.")

   OttoneuID  FG MajorLeagueID  Avg Salary  Median Salary  Min Salary  \
0  -1.414214         -1.414214   -1.414214      -1.414214   -1.414214   
1  -0.707107         -0.707107   -0.707107      -0.707107   -0.707107   
2   0.000000          0.000000    0.000000       0.000000    0.000000   
3   0.707107          0.707107    0.707107       0.707107    0.707107   
4   1.414214          1.414214    1.414214       1.414214    1.414214   

   Max Salary   Last 10   Roster%       ADP      rPTS  ...  POS_POS1  \
0   -1.414214 -1.414214 -1.414214 -1.414214 -1.414214  ...       1.0   
1   -0.707107 -0.707107 -0.707107 -0.707107 -0.707107  ...       0.0   
2    0.000000  0.000000  0.000000  0.000000  0.000000  ...       0.0   
3    0.707107  0.707107  0.707107  0.707107  0.707107  ...       0.0   
4    1.414214  1.414214  1.414214  1.414214  1.414214  ...       0.0   

   POS_POS2  POS_POS3  POS_POS4  POS_POS5  PlayerId_P1  PlayerId_P2  \
0       0.0       0.0       0.0       0.0          1.0   